In [1]:
import numpy as np
import dynesty as dn

In [2]:
#Two functions taken from dynesty code
#https://github.com/joshspeagle/dynesty
def _quantile(x, q, weights=None):
    """
    Compute (weighted) quantiles from an input set of samples.
    Parameters
    ----------
    x : `~numpy.ndarray` with shape (nsamps,)
        Input samples.
    q : `~numpy.ndarray` with shape (nquantiles,)
       The list of quantiles to compute from `[0., 1.]`.
    weights : `~numpy.ndarray` with shape (nsamps,), optional
        The associated weight from each sample.
    Returns
    -------
    quantiles : `~numpy.ndarray` with shape (nquantiles,) 
        The weighted sample quantiles computed at `q`.
    """

    # Initial check.
    x = np.atleast_1d(x)
    q = np.atleast_1d(q)

    # Quantile check.
    if np.any(q < 0.0) or np.any(q > 1.0):
        raise ValueError("Quantiles must be between 0. and 1.")

    if weights is None:
        # If no weights provided, this simply calls `np.percentile`.
        return np.percentile(x, list(100.0 * q))
    else:
        # If weights are provided, compute the weighted quantiles.
        weights = np.atleast_1d(weights)
        if len(x) != len(weights):
            raise ValueError("Dimension mismatch: len(weights) != len(x).")
        idx = np.argsort(x)  # sort samples
        sw = weights[idx]  # sort weights
        cdf = np.cumsum(sw)[:-1]  # compute CDF
        cdf /= cdf[-1]  # normalize CDF
        cdf = np.append(0, cdf)  # ensure proper span
        quantiles = np.interp(q, cdf, x[idx]).tolist()
    return quantiles

def quarts(results):
    samples = results['samples']
    try:
        weights = np.exp(results['logwt'] - results['logz'][-1])
    except:
        weights = results['weights']
    samples = np.atleast_1d(samples)
    if len(samples.shape) == 1:
        samples = np.atleast_2d(samples)
    else:
        assert len(samples.shape) == 2, "Samples must be 1- or 2-D."
        samples = samples.T
    med = []
    for i, x in enumerate(samples):
        #qm = _quantile(x, [0.5], weights=weights)
        qm = _quantile(x, [0.16, 0.5, 0.84], weights=weights)
        med.append(qm)
    return med

In [3]:
#Generate fake data 
def constant_data(noise, v_sys):
    return np.random.normal(v_sys,noise,100)

def var_data(noise, v_sys, v_dev):
    data = np.random.normal(v_sys,v_dev,100)
    for i in range(0,len(data)): 
        data[i] = np.random.normal(data[i],noise,1)
    return data

In [4]:
ndim_var = 2
ndim_constant = 1

#Functions to optimize
def test_var(x): 
    v_sys = x[0]
    s = x[1]
    return(-.5*(np.sum(np.log((rv_err_s**2)+(s**2)))+np.sum(((rv_s-v_sys)**2)/((rv_err_s**2)+(s**2)))))
    
def test_c(x):
    v_sys = x[0]
    return(-.5*(np.sum(np.log(rv_err_s**2))+np.sum(((rv_s-v_sys)**2)/(rv_err_s**2))))

#Uniform priors
def p_c(u): 
    v_sys = 400. * (2. * u - 1.)
    return v_sys

def p_v(u): 
    v_sys, s = u
    v_sys = 400. * (2. * v_sys - 1.)
    s =  25. * s 
    return v_sys, s

In [9]:
#Choose to produce data from variable+noise (var_data) or noise (constant_data) model
v_sys = 60
noise = 5
variability = 10
#rv_s = constant_data(noise, v_sys)
rv_s = var_data(noise,v_sys,variability)
rv_err_s = np.full(len(rv_s),noise)

#Calculate 
sampler_var = dn.NestedSampler(test_var, p_v, ndim_var) 
sampler_c = dn.NestedSampler(test_c, p_c, ndim_constant)
   
sampler_var.run_nested() 
results_var = sampler_var.results
logz_var = results_var['logz'][-1]
res_var = quarts(results_var)
    
sampler_c.run_nested()
results_c = sampler_c.results
logz_c = results_c['logz'][-1]
res_c = quarts(results_c)
    
#Print results
print('Results using variable + noise model:')
print('V_sys = %d, Variability = %d' % (res_var[0][1], res_var[1][1]))
print('Results using noise model:')
print('V_sys = %d' % res_c[0][1])

iter: 3626 | +500 | bound: 5 | nc: 1 | ncall: 21336 | eff(%): 19.338 | loglstar:   -inf < -405.608 <    inf | logz: -411.948 +/-  0.138 | dlogz:  0.001 >  0.509                                      

Results using variable + noise model:
V_sys = 59, Variability = 10
Results using noise model:
V_sys = 59
